In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Activation , Conv2D, Conv3D,Flatten , MaxPool2D,MaxPooling2D, Dropout
import os
import sys
from keras.utils import to_categorical
from keras.optimizers import rmsprop , adam, Adagrad
from keras.callbacks import EarlyStopping
from keras.layers import BatchNormalization
seed_value = 1234
os.environ['PYTHONHASHSEED']=str(seed_value)
import random
random.seed(seed_value)
import numpy as np
np.random.seed(seed_value)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data_dir = "/content/drive/My Drive/Colab Notebooks/"
sys.argv = []
sys.argv.append(data_dir+"cnn_a.py")
sys.argv.append(data_dir+"train.csv")
sys.argv.append(data_dir+"test.csv")
sys.argv.append(data_dir+"output.txt")
train = pd.read_csv(sys.argv[1],header=None)
test  = pd.read_csv(sys.argv[2],header=None)

In [0]:
train_data = train.iloc[:,:][0].str.split(expand=True)
train_data = train_data.astype(int)
test_data = test.iloc[:,:][0].str.split(expand=True)
test_data = test_data.astype(int)

In [9]:
k = 40000
val = 10000

X = train_data.iloc[:k,:-1].values
X_train = (X-np.mean(X,axis=0)).reshape(k,32,32,3)
Y_train = to_categorical(train_data.iloc[:k,-1].values).reshape(k,10) 

X = train_data.iloc[k:k+val,:-1].values
X_val = (X-np.mean(X,axis=0)).reshape(val,32,32,3)
Y_val = to_categorical(train_data.iloc[k:k+val,-1].values).reshape(val,10)

X = test_data.iloc[:,:-1].values
X_test = (X-np.mean(X,axis=0)).reshape(test_data.shape[0],32,32,3)

X_train = X_train/255
X_val = X_val/255
X_test = X_test/255

print(X_train.shape, Y_train.shape,X_val.shape,Y_val.shape, X_test.shape)

(40000, 32, 32, 3) (40000, 10) (10000, 32, 32, 3) (10000, 10) (10000, 32, 32, 3)


In [0]:
model = Sequential()
model.add(Conv2D(64, kernel_size=3, activation='relu',strides=1,padding="same", input_shape=(32,32,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(128, kernel_size=3, activation='relu',padding="same",strides=1))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(512,activation="relu"))
model.add(Dense(256,activation="relu"))
model.add(BatchNormalization())
model.add(Dense(10,activation="softmax"))
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto', baseline=None, restore_best_weights=True)
# keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)


In [56]:
optimizer = adam(lr=0.001, beta_1=0.9, beta_2=0.999)
# optimizer = Adagrad(lr=0.001, decay=1e-6,momentum=0.9)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, validation_data=(X_val, Y_val),callbacks = [],shuffle=True,epochs=30,batch_size=50)

Train on 40000 samples, validate on 10000 samples
Epoch 1/30
40000/40000 [==============================] - 14s 357us/step - loss: 1.3617 - acc: 0.5199 - val_loss: 1.3602 - val_acc: 0.5405
Epoch 2/30
40000/40000 [==============================] - 12s 302us/step - loss: 1.0692 - acc: 0.6259 - val_loss: 1.4925 - val_acc: 0.5209
Epoch 3/30
40000/40000 [==============================] - 12s 297us/step - loss: 0.9368 - acc: 0.6747 - val_loss: 2.0704 - val_acc: 0.4441
Epoch 4/30
40000/40000 [==============================] - 12s 295us/step - loss: 0.8349 - acc: 0.7114 - val_loss: 1.1865 - val_acc: 0.6105
Epoch 5/30
40000/40000 [==============================] - 12s 298us/step - loss: 0.7508 - acc: 0.7393 - val_loss: 1.1322 - val_acc: 0.6267
Epoch 6/30
40000/40000 [==============================] - 12s 298us/step - loss: 0.6661 - acc: 0.7659 - val_loss: 0.9801 - val_acc: 0.6723
Epoch 7/30
40000/40000 [==============================] - 12s 299us/step - loss: 0.5949 - acc: 0.7915 - val_loss: 1.

KeyboardInterrupt: ignored

In [0]:
Y_pred = model.predict_classes(X_test)
np.savetxt("predection.txt",Y_pred)

In [0]:
model = Sequential()
model.add(Conv2D(64, kernel_size=3, activation='relu',strides=1,padding="same", input_shape=(32,32,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(128, kernel_size=3, activation='relu',padding="same",strides=1))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(512,activation="relu"))
model.add(Dense(256,activation="relu"))
model.add(BatchNormalization())
model.add(Dense(10,activation="softmax"))
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=1, mode='auto', baseline=None, restore_best_weights=True)
# optimizer = adam(lr=0.0005, beta_1=0.9, beta_2=0.999)
optimizer = Adagrad(lr=0.001, decay=1e-6)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, validation_data=(X_val, Y_val),callbacks = [es],shuffle=True,epochs=30)